<a href="https://colab.research.google.com/github/ElenaMorozzova/NLP_Intro/blob/main/NLP_DZ5_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/
взять нер из nltk
проверить deeppavlov
написать свой нер попробовать разные подходы
передаём в сетку токен и его соседей
передаём в сетку только токен
сделать выводы по вашим экспериментам какой из подходов успешнее справляется


In [ ]:
# deeppavlov - после установки не удалось запустить build_model в колаб, выдавал ошибку:
# AttributeError: module 'numpy.linalg.lapack_lite' has no attribute '_ilp64'

#!pip uninstall -y tensorflow tensorflow-gpu
#!pip install numpy scipy librosa unidecode inflect librosa transformers
#!pip install deeppavlov

In [ ]:
!pip install corus

     |████████████████████████████████| 83 kB 2.1 MB/s 


In [ ]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-10-27 19:09:49--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  2.28MB/s    in 0.8s    

2021-10-27 19:09:50 (2.28 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [ ]:
!unzip collection5.zip

Archive:  collection5.zip
   creating: Collection5/
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.txt     
  inflating: Collection5/013.ann    

In [ ]:
!rm collection5.zip

In [ ]:
!ls

Collection5  sample_data


In [ ]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
rec = next(records)
rec

Ne5Markup(
    id='last_30_new',
    text='Галушка приступил к работе по ликвидации последствий наводнения (ДФО)\r\n\r\nВ район бедствия в Хабаровский край накануне вылетел новый министр по развитию Дальнего Востока Александр Галушка. Как сообщает корреспондент ИА REGNUM со ссылкой на сайт полномочного представителя президента РФ по Дальневосточному федеральному округу, Галушка намерен сам разобраться на месте, какой объем работы предстоит сделать в пострадавших районах.\r\n\r\nКак сообщало ранее ИА REGNUM, Александр Галушка назначен министром по развитию Дальнего Востока. Ранее Минвостокразвития возглавлял Виктор Ишаев. Однако в конце августа он был отстранен от должности. Информация о возможном назначении Галушки министром появилась в СМИ на прошлой неделе. Эксперты - как регионального, так и федерального уровня, высказывали недоумение относительно его кандидатуры. Прежде всего, у экспертов возникают сомнения относительно взаимодействия Галушки с полпредом в Дальневосточном округе Юр

In [ ]:
!pip install razdel

In [ ]:
from razdel import tokenize
import pandas as pd

In [ ]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
       
        result = 'None'        
        
        for item in rec.spans:            
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'PER'):
                result = 'PER'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'ORG'):
                result = 'ORG'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'MEDIA'):
                result = 'MEDIA'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'LOC'):
                result = 'LOC'
                break
            if (token.start >= item.start) and (token.stop <= item.stop) and (item.type == 'GEOPOLIT'):
                result = 'GEOPOLIT'
                break
                
    
        words.append([token.text, result])
    words_docs.extend(words)

In [ ]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [ ]:
df_words.head()

,word,tag
0,Д,PER
1,.,PER
2,Медведев,PER
3,уволил,None
4,двух,None


In [ ]:
df_words['tag'].value_counts()

None        219094
PER          21185
ORG          13649
LOC           4556
GEOPOLIT      4355
MEDIA         2477
Name: tag, dtype: int64

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model

In [ ]:
# разделяем датасет
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# кодируем целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
train_x.apply(len).max(axis=0)

55

In [ ]:
valid_x

49680     взаимосвязь
250351     предложено
185776              в
193899              и
5568             этом
             ...     
124307              в
83114         февраля
67249        отставку
209279          жизни
56626              14
Name: word, Length: 66329, dtype: object

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def custom_standardization(input_data):
    return input_data

def data_prep(train_data, seq_len=1, vocab_size = 30000):    
    
    vocab_size = 30000
    #seq_len = 1

    vectorize_layer = TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=seq_len)


    # Make a text-only dataset (no labels) and call adapt to build the vocabulary.
    text_data = train_data.map(lambda x, y: x)
    vectorize_layer.adapt(text_data)
    return vectorize_layer

In [ ]:
len(vectorize_layer.get_vocabulary())

29918

In [ ]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(len(df_words['tag'].value_counts()), activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        return self.fc3(concat_x)

In [ ]:
# Тренируем на 5 эпохах
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 1, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12437/12437 [==============================] - 237s 19ms/step - loss: 0.2826 - accuracy: 0.9181 - val_loss: 0.2056 - val_accuracy: 0.9400
Epoch 2/5
12437/12437 [==============================] - 231s 19ms/step - loss: 0.1214 - accuracy: 0.9634 - val_loss: 0.3450 - val_accuracy: 0.8951
Epoch 3/5
12437/12437 [==============================] - 231s 19ms/step - loss: 0.1078 - accuracy: 0.9657 - val_loss: 0.2934 - val_accuracy: 0.8956
Epoch 4/5
12437/12437 [==============================] - 232s 19ms/step - loss: 0.1032 - accuracy: 0.9666 - val_loss: 0.2730 - val_accuracy: 0.8949
Epoch 5/5
12437/12437 [==============================] - 232s 19ms/step - loss: 0.1009 - accuracy: 0.9671 - val_loss: 0.2828 - val_accuracy: 0.8949


In [ ]:
# Увеличим длину анализируемой последовательности - показатели качества на 5 эпохе чуть ниже, чем в первой модели
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 3, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12437/12437 [==============================] - 228s 18ms/step - loss: 0.2931 - accuracy: 0.9148 - val_loss: 0.2160 - val_accuracy: 0.9389
Epoch 2/5
12437/12437 [==============================] - 234s 19ms/step - loss: 0.1249 - accuracy: 0.9626 - val_loss: 0.3625 - val_accuracy: 0.8821
Epoch 3/5
12437/12437 [==============================] - 233s 19ms/step - loss: 0.1094 - accuracy: 0.9655 - val_loss: 0.2584 - val_accuracy: 0.9411
Epoch 4/5
12437/12437 [==============================] - 234s 19ms/step - loss: 0.1042 - accuracy: 0.9663 - val_loss: 0.2681 - val_accuracy: 0.8833
Epoch 5/5
12437/12437 [==============================] - 232s 19ms/step - loss: 0.1014 - accuracy: 0.9669 - val_loss: 0.2867 - val_accuracy: 0.8838


In [ ]:
# Добавим один дополнительный слой нейронов
class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc1 = Dense(150, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(len(df_words['tag'].value_counts()), activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        return self.fc3(concat_x)

In [ ]:
# Добавление одного слоя на 5 эпохах несколько улучшило качество
vocab_size = 30000
vectorize_layer = data_prep(train_data, seq_len = 1, vocab_size = vocab_size)


mmodel = modelNER()
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
mmodel.fit(train_data, validation_data=valid_data, epochs=5)

Epoch 1/5
12437/12437 [==============================] - 234s 19ms/step - loss: 0.2848 - accuracy: 0.9174 - val_loss: 0.2055 - val_accuracy: 0.9398
Epoch 2/5
12437/12437 [==============================] - 236s 19ms/step - loss: 0.1221 - accuracy: 0.9637 - val_loss: 0.2642 - val_accuracy: 0.8955
Epoch 3/5
12437/12437 [==============================] - 235s 19ms/step - loss: 0.1085 - accuracy: 0.9657 - val_loss: 0.2155 - val_accuracy: 0.9414
Epoch 4/5
12437/12437 [==============================] - 234s 19ms/step - loss: 0.1037 - accuracy: 0.9666 - val_loss: 0.2303 - val_accuracy: 0.9414
Epoch 5/5
12437/12437 [==============================] - 233s 19ms/step - loss: 0.1012 - accuracy: 0.9670 - val_loss: 0.2523 - val_accuracy: 0.8952
